In [ ]:

import torch
import torch.nn as nn
import numpy as np

# Define parameters
batch_size = 32
sequence_length = 100  # Number of time steps
vector_size = 16       # Features at each time step
embedding_dim = vector_size  # Embedding size matches vector_size

# Sample input tensor
inputs = torch.rand(batch_size, sequence_length, vector_size)  # [batch, seq_len, vector_size]

# Positional embeddings
positional_embeddings = nn.Embedding(sequence_length, embedding_dim)
positions = torch.arange(sequence_length).expand(batch_size, -1)  # [batch, seq_len]
pos_embeds = positional_embeddings(positions)  # [batch, seq_len, embedding_dim]

# Add positional embeddings to inputs
inputs_with_pos = inputs + pos_embeds

# Generate time-series specific embeddings
# Time-of-day and day-of-week features
time_of_day = torch.randint(0, 24, (batch_size, sequence_length))  # Hour of the day
day_of_week = torch.randint(0, 7, (batch_size, sequence_length))  # Day of the week

# Time embeddings for time-of-day and day-of-week
time_of_day_embedding = nn.Embedding(24, embedding_dim)
day_of_week_embedding = nn.Embedding(7, embedding_dim)

# Encode time features
time_of_day_embeds = time_of_day_embedding(time_of_day)  # [batch, seq_len, embedding_dim]
day_of_week_embeds = day_of_week_embedding(day_of_week)  # [batch, seq_len, embedding_dim]

# Combine time-of-day and day-of-week embeddings
time_series_embedding = time_of_day_embeds + day_of_week_embeds  # [batch, seq_len, embedding_dim]

# Add time differences as an additional feature
time_differences = torch.rand(batch_size, sequence_length)  # Simulate time differences
time_diff_embedding_layer = nn.Linear(1, embedding_dim)
time_diff_embeds = time_diff_embedding_layer(time_differences.unsqueeze(-1))  # [batch, seq_len, embedding_dim]

# Use frequency-domain features (e.g., Fourier transforms) to encode periodicity
frequency_features = torch.fft.rfft(inputs, dim=1).abs()  # Magnitude of Fourier transform
freq_embedding_layer = nn.Linear(frequency_features.shape[-1], embedding_dim)
freq_embeds = freq_embedding_layer(frequency_features)  # [batch, seq_len, embedding_dim]

# Introduce trainable parameters for embedding additional temporal events (e.g., holidays, anomalies)
# Assume binary indicators for holidays and anomalies
holiday_indicators = torch.randint(0, 2, (batch_size, sequence_length))  # Binary indicator
anomaly_indicators = torch.randint(0, 2, (batch_size, sequence_length))  # Binary indicator

holiday_embedding = nn.Embedding(2, embedding_dim)
anomaly_embedding = nn.Embedding(2, embedding_dim)

holiday_embeds = holiday_embedding(holiday_indicators)  # [batch, seq_len, embedding_dim]
anomaly_embeds = anomaly_embedding(anomaly_indicators)  # [batch, seq_len, embedding_dim]

# Convolutional layer to extract local patterns
class ConvFeatureExtractor(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size=3):
        super(ConvFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, output_dim, kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(output_dim, output_dim, kernel_size, padding=kernel_size//2)

    def forward(self, x):
        # Reshape input for Conv1d: [batch, vector_size, sequence_length]
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        # Reshape back to [batch, sequence_length, output_dim]
        return x.permute(0, 2, 1)

# Instantiate convolutional feature extractor
conv_extractor = ConvFeatureExtractor(input_dim=vector_size, output_dim=embedding_dim, kernel_size=3)

# Apply convolutional feature extraction
conv_features = conv_extractor(inputs)  # [batch, seq_len, embedding_dim]

# Combine all embeddings
final_inputs = (
    conv_features
    + inputs_with_pos
    + time_series_embedding
    + time_diff_embeds
    + freq_embeds
    + holiday_embeds
    + anomaly_embeds
)

print("Final input tensor shape after convolutions and embeddings:", final_inputs.shape)
